In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import moment
from numpy import linalg as LA
from sklearn.linear_model import Lasso, LassoLarsIC
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)
#Estimation of a Sparce Partial correlation matrix for a given data
class WeightMatrix:
    def __init__(self, Data, maxit, tolerance):

        self.D = Data
        self.maxit =maxit
        self.tol=tolerance
        self.var=Data.columns
        self.W=0

    def ConstructNet(self):
        N_variables=len(self.var)
        w=np.zeros((N_variables, N_variables), dtype=float)
        Data=np.array(self.D)
        for k in range(N_variables):
            X=np.delete(Data,k,1)
            y=Data[:,k]
            model = LassoLarsIC(criterion='aic')
            model.fit(X, y)
            dense_lasso = Lasso(alpha=model.alpha_, fit_intercept=True, max_iter=self.maxit, tol=self.tol)
            dense_lasso.fit(X, y)
            w[k,:]=np.insert(dense_lasso.coef_, k, 0)
        self.W = pd.DataFrame(w, columns=self.var, index=self.var)
        self.W=Directionality(self.W, self.D)
        self.W=AM_Normalization(self.W)
        return self.W
def Directionality(W, D):
    eps=0.0521 
    w=np.array(W); A=np.array(W)
    A[A!=0]=1; A=A+A.T; A[A!=2]=0
    for i in range(len(W.iloc[:,1])):
        A[i,0:i]=0
    index=np.transpose(np.nonzero(A))
    Genes=W.columns;
    M=pd.DataFrame(w, columns=Genes, index=Genes)
    for k in range(len(index)): 
        [x,y]=np.array(index[k])
        if abs(W.iloc[x,y])>eps and abs(W.iloc[y,x])>eps:
            Wx=w[x,:]; Wx[y]=0; X=D.iloc[:,x]-D.dot(Wx)
            Wy=w[y,:]; Wy[x]=0; Y=D.iloc[:,y]-D.dot(Wy)
            GamX=abs(moment(X, moment=3)/pow(stats.tstd(X),3))
            GamY=abs(moment(Y, moment=3)/pow(stats.tstd(Y),3))
            DelX=abs(moment(X, moment=4)/pow(stats.tstd(X),4)-3)
            DelY=abs(moment(Y, moment=4)/pow(stats.tstd(Y),4)-3)
            if abs(W.iloc[y,x])>=abs(W.iloc[x,y]) and GamX>GamY and DelX>DelY:
                M.iloc[x,y]=0
            elif abs(W.iloc[y,x])<=abs(W.iloc[x,y]) and GamX<GamY and DelX<DelY:
                M.iloc[y,x]=0
        elif abs(W.iloc[x,y])>eps and abs(W.iloc[y,x])<=eps:  
            M.iloc[y,x]=0
        elif abs(W.iloc[x,y])<=eps and abs(W.iloc[y,x])>eps: 
            M.iloc[x,y]=0
    return M

def Inside_L1ball(v): 
    n, = v.shape  
    s = np.abs(v)
    if s.sum() <= 1:
        return v
    else:
        u = np.sort(s)[::-1]
        cSum = np.cumsum(u)
        rho = np.nonzero(u * np.arange(1, n+1) > (cSum - 1))[0][-1]
        theta = float(cSum[rho] - 1) / (rho+1)
        x = (s - theta).clip(min=0)
        x *= np.sign(v)
    return x

def AM_Normalization(W):
    NormW=0; k=0
    while NormW<LA.norm(W) or k<1:
        for i in range(W.shape[0]):
            W.iloc[i,:]=Inside_L1ball(np.array(W.iloc[i,:]))
        for i in range(W.shape[1]):
            W.iloc[:,i]=Inside_L1ball(np.array(W.iloc[:,i]))
        NormW=LA.norm(W)
        k=k+1
    return W

In [2]:
maxit=10000; tolerance=1e-12; 
ExpData = pd.read_csv('EXP_DATA_LUAD.csv', index_col=0)  
SigData = pd.read_csv('SIG_DATA_LUAD.csv', index_col=0)
D=pd.concat([ExpData, SigData], axis=1)
Net=WeightMatrix(D, maxit, tolerance)  
Weight_Matrix=Net.ConstructNet()
Weight_Matrix.to_csv('Weight_Matrix-LUAD.csv')

In [5]:
from pyvis.network import Network

def VisualizeNetwork(W, Signatures, th):
    net=Network(height='800px', width='100%', bgcolor='white', font_color='black', directed = True)
    net.barnes_hut()

    sources=[]; targets=[]; weights=[]
    for sig in Signatures:
        t=W.loc[:,sig]
        t=t[abs(t)>th]
        r=W.loc[sig,:]
        r=r[abs(r)>th]
        sources =sources+[sig] * len(t)+list(r.index)
        targets=targets+list(t.index)+[sig] * (len(r))
        weights=weights+list(t)+list(r)
    edges=zip(sources,targets, weights)

    net.toggle_physics(False)
    for e in edges:
        reg = e[0]
        tar=e[1]
        w=e[2]
        if reg in Signatures:
            net.add_node(reg, reg, title=reg, color='orange', shape = 'owl')
        else:
            net.add_node(reg, reg, title=reg, color='green')
        if tar in Signatures:
            net.add_node(tar, tar, title=tar, color='orange', shape = 'owl')
        else:
            net.add_node(tar, tar, title=reg, color='green')    
        if w>0:
            net.add_edge(reg, tar, value=w, color='red')
        else:
            net.add_edge(reg, tar, value=w, color='blue')
    
    neighbor_map = net.get_adj_list()
    for node in net.nodes:
        node['title'] += ' Neighbor:<>'+'<br>'.join(neighbor_map[node['id']])
        node['value'] = len(neighbor_map[node['id']])
    net.show('SignatureNetwork.html')
Signatures=SigData.columns
th=0.02
VisualizeNetwork(Weight_Matrix, Signatures, th)

In [6]:
Signatures

Index(['SBS1', 'SBS2', 'SBS4', 'SBS5', 'SBS13', 'SBS40'], dtype='object')

In [12]:
Weight_Matrix

,TNFRSF1A,RAD54L,SMAD6,IL1A,GPR65,TAP2,FGFR4,IL10RB,ERI3,ARTN,...,SBS4,SBS5,SBS13,SBS40,SBS1,SBS2,SBS4,SBS5,SBS13,SBS40
TNFRSF1A,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
RAD54L,-0.004865,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.07779,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
SMAD6,0.003333,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
IL1A,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
GPR65,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.0,...,0.0,0.000000,-0.000000,0.0,0.000000,0.000000,0.0,0.0,-0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SBS2,0.000000,0.085368,0.000000,0.000000,-0.0,0.0,0.0,0.027744,0.00000,0.0,...,-0.0,0.000000,-0.000000,-0.0,-0.000000,0.000000,0.0,0.0,0.0,-0.0
SBS4,0.000000,0.166942,-0.029649,-0.078699,-0.0,0.0,0.0,0.000000,-0.00000,0.0,...,0.0,-0.000000,-0.000000,-0.0,-0.005994,0.000000,0.0,0.0,0.0,0.0
SBS5,0.000000,0.004805,0.000000,-0.009877,-0.0,-0.0,0.0,0.000000,0.00000,0.0,...,0.0,0.000000,-0.050602,-0.0,-0.000000,-0.000000,-0.0,0.0,0.0,0.0
SBS13,0.000000,0.162491,0.000000,0.000000,0.0,0.0,0.0,0.002848,0.00000,0.0,...,0.0,0.000000,0.000000,-0.0,-0.002562,-0.021956,0.0,-0.0,0.0,0.0
